## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [3]:
import torch.nn as nn
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch

we represent the inputs and targets and matrices.

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [6]:
from torch.utils.data import TensorDataset

In [7]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [8]:
from torch.utils.data import DataLoader

In [9]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [10]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 73.,  66.,  44.],
        [101.,  44.,  37.],
        [ 91.,  88.,  64.],
        [ 74.,  66.,  43.]])
tensor([[ 22.,  37.],
        [ 57.,  69.],
        [ 21.,  38.],
        [ 81., 101.],
        [ 57.,  69.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [11]:
# Define model
model = nn.Linear(3, 2) # 3 input two output
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3025, -0.0189, -0.1634],
        [-0.2218,  0.3197,  0.0163]], requires_grad=True)
Parameter containing:
tensor([-0.4094, -0.4494], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [12]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3025, -0.0189, -0.1634],
         [-0.2218,  0.3197,  0.0163]], requires_grad=True),
 Parameter containing:
 tensor([-0.4094, -0.4494], requires_grad=True)]

In [13]:
# Generate predictions
preds = model(inputs)
preds

tensor([[13.3774,  5.4802],
        [14.9936,  8.5443],
        [13.8930, 24.0411],
        [23.5835, -8.7246],
        [ 7.2077, 16.0806],
        [13.6988,  4.9386],
        [14.8491,  8.2409],
        [14.0321, 23.8356],
        [23.2621, -8.1830],
        [ 6.7419, 16.3188],
        [13.2330,  5.1768],
        [15.3150,  8.0028],
        [14.0374, 24.3445],
        [24.0494, -8.9628],
        [ 6.8863, 16.6222]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [14]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities.

In [15]:
# Define loss function
loss_fn = F.mse_loss

In [16]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6360.4136, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [17]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.


In [18]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [19]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 305.4825
Epoch [20/100], Loss: 209.0381
Epoch [30/100], Loss: 357.5923
Epoch [40/100], Loss: 136.5826
Epoch [50/100], Loss: 69.2558
Epoch [60/100], Loss: 36.5623
Epoch [70/100], Loss: 9.3067
Epoch [80/100], Loss: 28.9175
Epoch [90/100], Loss: 23.9548
Epoch [100/100], Loss: 36.8141


In [20]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.4235,  71.1764],
        [ 79.7421,  97.7275],
        [120.2343, 137.9842],
        [ 30.0800,  42.3500],
        [ 92.4479, 110.8371],
        [ 57.3463,  70.1179],
        [ 79.1098,  97.2620],
        [120.3244, 138.3061],
        [ 31.1572,  43.4085],
        [ 92.8928, 111.4300],
        [ 57.7912,  70.7108],
        [ 78.6649,  96.6690],
        [120.8665, 138.4498],
        [ 29.6351,  41.7571],
        [ 93.5251, 111.8956]], grad_fn=<AddmmBackward0>)

In [21]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [22]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.7372, 67.6708]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.7 tons per hectare, and that of oranges is 67.6 tons per hectare.